In [15]:
import requests
import re
from bs4 import BeautifulSoup as bs
import time
from pprint import pprint
import pickle

In [2]:
from transliterate import translit, get_available_language_codes

In [24]:
def clear_string(string):
    if string:
        return(re.sub('[-|.|,|"|/|\xa0|\u202f| ]', '', string).lower())
    else:
        return None

In [3]:
def salary_parser(salary):
    
    salary_dict = {
        'min':None,
        'max':None,
        'currency':None,
        'comment':None
    }
    
    if salary is None:
        return salary_dict
    
    salary = salary.replace('\xa0', '')
    salary = salary.replace('\u202f', '')
    
    
    if 'от ' in salary.lower():
        salary_dict['comment'] = 'От'
        salary = salary.replace(' ', '')
        salary_dict['min'] = re.search('[0-9]+', salary)[0]
    elif 'до ' in salary.lower():
        salary_dict['comment'] = 'До'
        salary = salary.replace(' ', '')
        salary_dict['max'] = re.search('[0-9]+', salary)[0]
    elif '—' in salary:
        salary_dict['currency'] = salary.split()[-1]
        salary = salary.replace(' ','')
        search = re.findall('[0-9]+', salary)
        salary_dict['min'], salary_dict['max'] = search[0], search[1]
    else:
        
        if re.findall('[0-9]+', salary):
            salary = salary.replace(' ','')
            search = re.findall('[0-9]+', salary)
            salary_dict['min'], salary_dict['max'] = search[0], search[0]
        else:
            salary_dict['comment'] = salary
    
    return salary_dict

In [25]:
def job_searcher_rabota_ru(query, max_pages=20):
    """
    Функция по поиску работы на работа.ру
       : param query : Желаемая должность
       : param max_pages : Максимальное количество страниц, которое запросит алгоритм
    """
    
    
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
    }
    url = 'https://nn.rabota.ru/vacancy'
    page = 1
    results = []

    while True:
        
        if page > max_pages:
            break
        params = {
            'query' : query,
            'sort' : 'relevance',
            'page' : page
        }
        response = requests.get(url, params, headers=headers)
        
        print(f'Выполняется запрос, страница: {page}')
        
        if not response.ok:
            break

        soup = bs(response.text, 'html.parser')

        vacancies = soup.find_all('h3', attrs={'class':'vacancy-preview-card__title'})
        if not vacancies:
            break

        platform = url.split(sep='/')[2]

        for vacancy in vacancies:
            title = vacancy.find('a').text.strip()
            salary = vacancy.parent.find('div', {'class':'vacancy-preview-card__salary'}).find('a').text.strip()
            ref = url + vacancy.find('a')['href']
            try:
                employer = vacancy.parent.parent.parent.find('span', {'class':'vacancy-preview-card__company-name'}).find('a').text.strip()
            except AttributeError:
                employer = None
            results.append({
                'ref':ref,
                'salary_raw':salary,
                'salary':salary_parser(salary),
                'title':title,
                'platform':platform,
                'employer':employer,
                'clear_title':clear_string(title),
                'clear_employer':clear_string(employer)
            })
        page += 1
        time.sleep(3)
    print('Выполнение завершено.')    
    return results

In [39]:
def job_searcher_hh_ru(query, max_pages=20):
    """
    Функция по поиску работы на работа.ру
       : param query : Желаемая должность
       : param max_pages : Максимальное количество страниц, которое запросит алгоритм
    """    
    
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
    }
    url = 'https://nn.hh.ru/vacancies/' + translit(query, 'ru', reversed=True)
    print(url)
    page = 1
    results = []
    while True:
        params = {
            'page' : page,
            'hhtmFrom' : 'vacancy_search_catalog',    
        }
        
        if page > max_pages:
            break
        response = requests.get(url, params, headers=headers)
        
        print(f'Выполняется запрос, страница: {page}')
        
        if not response.ok:
            break
        soup = bs(response.text, 'html.parser')

        vacancies = soup.find_all('div', attrs={'class':'vacancy-serp-item-body__main-info'})
        if not vacancies:
            break

        platform = url.split(sep='/')[2]

        for vacancy in vacancies:
            title = vacancy.find('a', {'class':'serp-item__title'}).text.strip()
            try:
                employer = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-employer'}).text.strip()
            except AttributeError:
                employer = None
            try:
                salary = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'}).text.strip()
            except AttributeError:
                salary = None
            ref = vacancy.find('a', {'class':'serp-item__title'})['href']
            results.append({
                'ref':ref,
                'salary_raw':salary,
                'salary':salary_parser(salary),
                'title':title,
                'platform':platform,
                'employer':employer,
                'clear_title':clear_string(title),
                'clear_employer':clear_string(employer)
            })
        page += 1
        time.sleep(3)
    print('Выполнение завершено.')    
    return results

In [40]:
vacancies_hh = job_searcher_hh_ru('программист', 10)

https://nn.hh.ru/vacancies/programmist
Выполняется запрос, страница: 1
Выполняется запрос, страница: 2
Выполняется запрос, страница: 3
Выполняется запрос, страница: 4
Выполняется запрос, страница: 5
Выполняется запрос, страница: 6
Выполняется запрос, страница: 7
Выполняется запрос, страница: 8
Выполняется запрос, страница: 9
Выполняется запрос, страница: 10
Выполнение завершено.


In [7]:
pprint(vacancies_hh)

[{'employer': 'ПАО\xa0Ростелеком',
  'platform': 'nn.hh.ru',
  'ref': 'https://nn.hh.ru/vacancy/69772460?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
  'salary': {'comment': 'От', 'currency': None, 'max': None, 'min': '50000'},
  'salary_raw': 'от 50\u202f000 руб.',
  'title': 'Аналитик данных'},
 {'employer': 'ООО\xa0Новые Промышленные Технологии',
  'platform': 'nn.hh.ru',
  'ref': 'https://nn.hh.ru/vacancy/69014257?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
  'salary': {'comment': None, 'currency': None, 'max': None, 'min': None},
  'salary_raw': None,
  'title': 'Маркетолог- аналитик'},
 {'employer': 'МАГНИТ, Розничная сеть',
  'platform': 'nn.hh.ru',
  'ref': 'https://nn.hh.ru/vacancy/54171010?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
  'salary': {'comment': None, 'currency': None, 'max': None, 'min': None},
  'salary_raw': None,
  'title': 'Бизнес-аналитик'},
 {'employer': 'АО\xa0НоваКард',
  'platform': 'nn.hh.ru',
  'ref': 'https://nn.hh

In [30]:
len(vacancies_hh)

100

In [33]:
len(vacancies)

23

In [41]:
vacancies = job_searcher_rabota_ru('программист', 10)

Выполняется запрос, страница: 1
Выполняется запрос, страница: 2
Выполняется запрос, страница: 3
Выполняется запрос, страница: 4
Выполняется запрос, страница: 5
Выполняется запрос, страница: 6
Выполнение завершено.


In [13]:
for vacancy in vacancies:
    print(vacancy['title'])
    print(vacancy['salary'])
    print(vacancy['salary_raw'])
    print(vacancy['ref'])
    print(vacancy['platform'])
    print('_'*25)

Аналитик
{'min': '50000', 'max': None, 'currency': None, 'comment': 'От'}
от 50 000 руб.
https://nn.rabota.ru/vacancy/vacancy/46906249/?search_id=166377709692655s42tnf7nb
nn.rabota.ru
_________________________
Специалист по опросу населения
{'min': '10000', 'max': '23000', 'currency': 'руб.', 'comment': None}
10 000 — 23 000 руб.
https://nn.rabota.ru/vacancy/vacancy/45497243/?search_id=166377709692655s42tnf7nb
nn.rabota.ru
_________________________
Аналитик бизнес-сопровождения продуктов управленческой отчетности
{'min': None, 'max': None, 'currency': None, 'comment': 'по договоренности'}
по договоренности
https://nn.rabota.ru/vacancy/vacancy/46900409/?search_id=166377709692655s42tnf7nb
nn.rabota.ru
_________________________
Бизнес-аналитик Creatio
{'min': None, 'max': None, 'currency': None, 'comment': 'по договоренности'}
по договоренности
https://nn.rabota.ru/vacancy/vacancy/46899259/?search_id=166377709692655s42tnf7nb
nn.rabota.ru
_________________________
Аналитик 1С
{'min': None,

In [27]:
pprint(vacancies)

[{'clear_employer': 'сетьювелирныхсалоновсенат',
  'clear_title': 'аналитик',
  'employer': 'Сеть ювелирных салонов "Сенат"',
  'platform': 'nn.rabota.ru',
  'ref': 'https://nn.rabota.ru/vacancy/vacancy/46906249/?search_id=1663778347670j7l0myf8vg',
  'salary': {'comment': 'От', 'currency': None, 'max': None, 'min': '50000'},
  'salary_raw': 'от 50\xa0000 руб.',
  'title': 'Аналитик'},
 {'clear_employer': 'агентствоисследованийиразработокооо',
  'clear_title': 'специалистпоопросунаселения',
  'employer': 'Агентство исследований и разработок ООО',
  'platform': 'nn.rabota.ru',
  'ref': 'https://nn.rabota.ru/vacancy/vacancy/45497243/?search_id=1663778347670j7l0myf8vg',
  'salary': {'comment': None,
             'currency': 'руб.',
             'max': '23000',
             'min': '10000'},
  'salary_raw': '10\xa0000\xa0— 23\xa0000 руб.',
  'title': 'Специалист по опросу населения'},
 {'clear_employer': 'сбер',
  'clear_title': 'аналитикбизнессопровожденияпродуктовуправленческойотчетности',

In [18]:
pprint(vacancies_hh)

[{'employer': 'ПАО\xa0Ростелеком',
  'platform': 'nn.hh.ru',
  'ref': 'https://nn.hh.ru/vacancy/69772460?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
  'salary': {'comment': 'От', 'currency': None, 'max': None, 'min': '50000'},
  'salary_raw': 'от 50\u202f000 руб.',
  'title': 'Аналитик данных'},
 {'employer': 'ООО\xa0Новые Промышленные Технологии',
  'platform': 'nn.hh.ru',
  'ref': 'https://nn.hh.ru/vacancy/69014257?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
  'salary': {'comment': None, 'currency': None, 'max': None, 'min': None},
  'salary_raw': None,
  'title': 'Маркетолог- аналитик'},
 {'employer': 'МАГНИТ, Розничная сеть',
  'platform': 'nn.hh.ru',
  'ref': 'https://nn.hh.ru/vacancy/54171010?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
  'salary': {'comment': None, 'currency': None, 'max': None, 'min': None},
  'salary_raw': None,
  'title': 'Бизнес-аналитик'},
 {'employer': 'АО\xa0НоваКард',
  'platform': 'nn.hh.ru',
  'ref': 'https://nn.hh

In [42]:
with open('../Les4/vacancies_rabota_ru', 'wb') as f:
    pickle.dump(vacancies, f)

In [43]:
with open('../Les4/vacancies_hh_ru', 'wb') as f:
    pickle.dump(vacancies_hh, f)

In [22]:
re.sub('[\xa0|еле]', '', 'ПАО\xa0Ростелеком')

'ПАОРостком'